In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os

In [ ]:
# a. Load the dataset
# Assuming you have the BCCD dataset images organized in directories by class
# Dataset structure:
# BCCD/
# ├── train/
# │   ├── WBC/
# │   ├── RBC/
# │   └── Platelets/
# └── val/
#     ├── WBC/
#     ├── RBC/
#     └── Platelets/

dataset_dir = 'path_to_your_dataset/BCCD'
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

In [ ]:
# b. Show the number of training and testing images
num_train_images = sum([len(files) for r, d, files in os.walk(train_dir)])
num_val_images = sum([len(files) for r, d, files in os.walk(val_dir)])

print(f"Number of training images: {num_train_images}")
print(f"Number of validation images: {num_val_images}")

In [ ]:
# c. Plot some images from the training dataset
def plot_images_from_directory(directory, num_images=9):
    plt.figure(figsize=(10, 10))
    for i, (root, _, files) in enumerate(os.walk(directory)):
        for j, file in enumerate(files):
            if j >= num_images:
                break
            img = plt.imread(os.path.join(root, file))
            ax = plt.subplot(3, 3, j + 1)
            plt.imshow(img)
            plt.title(root.split('/')[-1])
            plt.axis("off")
        if i >= 2:  # Limit to first 3 directories (classes)
            break

plot_images_from_directory(train_dir)

In [ ]:
# d. Image augmentation – contrast, flipping, and rotation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,       # Rotation
    horizontal_flip=True,    # Horizontal flipping
    brightness_range=[0.8, 1.2]  # Contrast adjustment
)

val_datagen = ImageDataGenerator(rescale=1.0/255)  # Just rescaling for validation

# Loading augmented images from the directories
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
# e. After augmentation, show the number of training and testing images
# The number of images remains the same in terms of actual dataset size but new augmented versions are generated on-the-fly
print(f"Number of augmented training images (on-the-fly): {num_train_images}")
print(f"Number of augmented validation images (on-the-fly): {num_val_images}")

In [ ]:
# f. Normalizing the training data is already handled in the ImageDataGenerator with `rescale=1.0/255`

In [ ]:
# g. Build a convolutional neural network for initial training without augmentation
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')  # 3 classes: WBC, RBC, Platelets
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the CNN model (without augmentation)
history = cnn_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

In [ ]:
# h. Show the training and testing accuracy (without augmentation)
plt.figure(figsize=(8, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy (No Augmentation)')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy (No Augmentation)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Testing Accuracy (No Augmentation)')
plt.show()

In [ ]:
# i. Build a convolutional neural network to train images with augmentation
# We'll use the same model architecture but train with augmented images this time

augmented_history = cnn_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10
)

In [ ]:
# j. Show the training and testing accuracy (with augmentation)
plt.figure(figsize=(8, 5))
plt.plot(augmented_history.history['accuracy'], label='Training Accuracy (With Augmentation)')
plt.plot(augmented_history.history['val_accuracy'], label='Validation Accuracy (With Augmentation)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Testing Accuracy (With Augmentation)')
plt.show()

In [ ]:
# k. Compare the training and testing accuracy before and after augmentation
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy (No Augmentation)')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy (No Augmentation)')
plt.plot(augmented_history.history['accuracy'], label='Training Accuracy (With Augmentation)')
plt.plot(augmented_history.history['val_accuracy'], label='Validation Accuracy (With Augmentation)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Comparison of Training and Testing Accuracy')
plt.show()